In [44]:
from langchain.schema import HumanMessage
from langchain_ollama import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

In [45]:
loader = PyPDFLoader("document01.pdf")
documents = loader.load()

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [47]:
print(chunks.__len__())

105


In [48]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [49]:
vectorstore = InMemoryVectorStore.from_texts(
    texts=[doc.page_content for doc in chunks],
    embedding=embeddings,
)

In [50]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

In [51]:
GROQ_API_KEY = "[GROQ_API_KEY]"
llm_compression = ChatGroq(temperature=0,api_key=GROQ_API_KEY,model="llama-3.3-70b-versatile")
compressor_llm = LLMChainExtractor.from_llm(llm_compression)
advanced_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_llm,
    base_retriever=base_retriever
)

In [52]:
def retrieve_context(query):
    """Récupère le contexte pertinent pour la question"""
    docs = advanced_retriever.get_relevant_documents(query)
    return "\n\n".join([f"- {doc.page_content}" for doc in docs])

# --- Configuration du prompt RAG ---
rag_template = """**Rôle** : Expert in analysing repport .
**Mission** : Giving though in how to improve the repport.

**Historique** :
{history}

**Connaissances** :
{context}

**Question** :
{question}

**Consignes** :
- Répondez uniquement avec les informations disponibles
- En cas de hors-sujet : "Je n'ai pas cette information. Voulez-vous une question sur les tendances énergétiques marocaines ?"
- Ton professionnel et clair

**Format** :
Structure claire avec informations factuelles
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)


llm_response = ChatGroq(temperature=0,model="llama-3.3-70b-versatile",api_key=GROQ_API_KEY)


memory = ConversationBufferMemory(memory_key="history", return_messages=False)

def generate_response(user_message):
    """Génère la réponse de l'assistant"""

    conversation_history = memory.buffer


    context = retrieve_context(user_message)


    prompt = rag_prompt.format(
        history=conversation_history,
        context=context,
        question=user_message
    )


    response = llm_response([HumanMessage(content=prompt)]).content


    memory.save_context({"input": user_message}, {"output": response})

    return response

In [53]:
import gradio as gr

In [54]:
with gr.Blocks() as demo:
    gr.Markdown("## Djnago ")
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt_input = gr.Textbox(
            show_label=False,
            placeholder="Posez votre question ici...",
            container=False
        )
        send_btn = gr.Button("Envoyer")

    def user_interaction(user_message, history):
        response = generate_response(user_message)
        history.append((user_message, response))
        return "", history

    txt_input.submit(user_interaction, [txt_input, state], [txt_input, chatbot])
    send_btn.click(user_interaction, [txt_input, state], [txt_input, chatbot])

demo.launch()

/tmp/ipykernel_83920/1037708201.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
